<h1>Data Processing</h1>

<h3>Importpath</h3>

Add Projectfolder to Importpath

In [ ]:
import sys
import os
cwd = os.path.abspath(".")
sys.path.insert(0, cwd.replace("Notebook", "Code"))

sys.path

<h3>Imports</h3>

In [ ]:
import pandas as pd

import fmp
import influx as db

<h3>Data Processing</h3>

In [ ]:
start_date = pd.Timestamp("2024-01-01")
stop_date = pd.Timestamp("2024-01-07")

FMP = fmp.FinancialModelingPrep(deployment = "linode")
FMP.economic_calendar_pipeline(start_date, stop_date)

In [ ]:
FMP = fmp.FinancialModelingPrep(deployment = "linode")
FMP.economic_calendar_pipeline_longrange()

<h3>Test new Configuration for InfluxDB</h3>

In [ ]:
import influx as db

db.InfluxDatabase()

<h3>Data Processing: Pricedata form MetaTrader5 CSV's to InfluxDB</h3>

In [ ]:
import influx
import mt5

db = influx.InfluxDatabase(deployment = "linode")
mt5 = mt5.MetaTrader5()

majors = ["AUDUSD", "EURUSD", "GBPUSD", "NZDUSD", "USDCAD", "USDCHF", "USDJPY"]

major = majors[1]

print("Step #1")
raw_data = mt5.load_csv_to_dataframe(file_name=f"MetaTrader5 {major} 1MIN Full Year 2023.csv")
print("Step #2")
clean_data = mt5.preprocess_csv_dataframe(raw_data, symbol = major, timezone="EST")

print(clean_data)

#print("Step #3")
#db.ingest_data(clean_data, mode="live")


<h3>Data Quering: InfluxDB Pricedata<h3>

In [ ]:
import influx
import pandas as pd

db = influx.InfluxDatabase()

majors = ["AUDUSD", "EURUSD", "GBPUSD", "NZDUSD", "USDCAD", "USDCHF", "USDJPY"]
major = majors[1]

time1 = pd.Timestamp("2023-06-23T16:12")

#AUDUSD 2023.06.23	16:12:00

data1 = db.query_data(time = time1, symbol = major, timeframe = "1min", source = "MetaTrader5")

print(data1)

### Test Trademade API

In [ ]:
import provider as pv
import pandas as pd

In [ ]:
provider = pv.Provider(deployment = "local")

In [ ]:
timestamp = pd.Timestamp("2024-02-15T13:26")
data = provider.foreign_exchange_rate_trademade(timestamp, "EURUSD", "close")
data

### Data Processing: ForexTester.com

In [ ]:
#import influx
import wrangler
import pandas as pd

#db = influx.InfluxDatabase(deployment = "linode")
source = wrangler.ForexTester()

majors = ["AUDUSD", "EURUSD", "GBPUSD", "NZDUSD", "USDCAD", "USDCHF", "USDJPY"]

major = majors[1]

source_name = "ForexTester.com"
timezone = "GMT"
timeframe = "1MIN"
source_file_name = f"{source_name} {major} {timeframe} {timezone} 2001-01-01 2024-01-31.txt"
sink_file_name = f"{source_name} {major} {timeframe} Full Year 2023.csv"
start_date = pd.Timestamp("2023-01-01")
end_date = pd.Timestamp("2024-01-01")

print("Step #1 Raw Data")
raw_data = source.load_csv_to_dataframe(file_name=source_file_name)

print("Strep 1B Short Data")
short_data = raw_data.iloc[7747870:]

print("Step #2 Intermediate Data")
intermediate_data = source.process_timeframe_csv_dataframe(short_data).loc[start_date:end_date]

print("Step #3 Sink Data")
source.populate_csv_file(intermediate_data, file_name=sink_file_name)

#print("Step #3 Clean Data")
#clean_data = source.preprocess_csv_dataframe(intermediate_data, symbol = major, timezone=timezone)

#print(clean_data)

#print("Step #3")
#db.ingest_data(clean_data, mode="live")

In [ ]:
import influx
import wrangler
import pandas as pd

db = influx.InfluxDatabase(deployment = "linode")
source = wrangler.ForexTester()

majors = ["AUDUSD", "EURUSD", "GBPUSD", "NZDUSD", "USDCAD", "USDCHF", "USDJPY"]

major = majors[1]

source_name = "ForexTester.com"
timezone = "GMT"
timeframe = "1MIN"
source_file_name = f"{source_name} {major} {timeframe} Full Year 2023.csv"

"ForexTester.com EURUSD 1MIN Full Year 2023.csv"

print("Step #1 Raw Data")
raw_data = source.load_csv_to_dataframe(file_name=source_file_name)

print("Step #2 Clean Data")
clean_data = source.preprocess_csv_dataframe(raw_data, symbol = major, timezone=timezone)

print(clean_data)

print("Step #3")
db.ingest_data(clean_data, mode="live")